In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

from langchain_google_community import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()

def top_results(query):
    return search.results(query, 5)

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=top_results,
)
# results = (tool.run("github url for CVE-2023-32315"))
# results

In [3]:
import fire
from dotenv import load_dotenv
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.tools import Tool
from exploiter.query.query_text_file import read_read_me, read_dockerfile, clone_repo
from exploiter.query.query_input_content import get_username_and_password, summarize_content
from exploiter.agent_action.python_executor import python_agent_executor
from exploiter.prompts import prompts, task_prompts


load_dotenv()
cve_number = "CVE-2023-32315"

def action_to_dict(action, observation):
    log_lines = action.log.split('\n')
    
    thought = log_lines[0].replace('Thought: ', '') if len(log_lines) > 0 else None
    action_name = log_lines[1].replace('Action: ', '') if len(log_lines) > 1 else None
    action_input = log_lines[2].replace('Action Input: ', '') if len(log_lines) > 2 else None
    
    return {
        "Thought": thought,
        "Action": action_name,
        "Action Input": action_input,
        "Observation": observation.strip()  # This will remove leading and trailing newlines from the message.
    }

base_prompt = hub.pull("langchain-ai/react-agent-template")

##### ROUNTER AGENT #####
def python_agent_executor_wrapper(original_prompt):
    return python_agent_executor.invoke({"input": original_prompt})

tools = [
    Tool(
        name="Python Agent",
        func=python_agent_executor_wrapper,
        description="""useful when you need to transform natual language to python and execute the python code,
                       returning the results of the code execution
                       DO NOT ACCEPT CODE AS INPUT""",
    ),
    Tool(
        name="Information Extraction Agent",
        func=get_username_and_password,
        description="""useful when you need to extract username and password""",
    ),
    Tool(
        name="READ_ME AGENT",
        func=read_read_me,
        description="""useful when you need to read a readme file given a parameter data.\
            Data is a dict with path: str type and extract_type: str type where value either 'usage' or 'guideline'""",
    ),
    Tool(
        name="DOCKERFILE AGENT",
        func=read_dockerfile,
        description="""useful when you need to read a dockerfile given app_name""",
    ),
    Tool(
        name="Summarize content",
        func=summarize_content,
        description="""useful when you need to summarize content""",
    ),
    Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=top_results,
    ),
        Tool(
            name="Clone a repo given an url",
            func=clone_repo,
            description="""useful when you need to clone a repo""",
        )
]
prompt = base_prompt.partial(instructions="")
router_agent = create_react_agent(
    prompt=prompt, llm=ChatOpenAI(temperature=0, model='gpt-3.5-turbo'), tools=tools
)
router_agent_executor = AgentExecutor(agent=router_agent, tools=tools, verbose=True, 
                                      return_intermediate_steps=True)

prompt = prompts[cve_number]
task_prompt = task_prompts[prompt['task']]


Start


In [7]:

response = router_agent_executor.invoke(
        {"input": f"Given the CVE name: {cve_number}.Follow below steps to give me my answer\
                    1. Search 'github url for {cve_number}'. \
                    2. Provide me first URL link. For example: https://github.com/walnutsecurity/cve-2021-42013\
                    3. Clone the repo \
        "}
    )

# results = {}
# results['steps'] = [action_to_dict(action, message) for action, message in response["intermediate_steps"]]
# results['final_output'] = response['output']
# print(results)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: google_search
Action Input: github url for CVE-2023-32315[{'title': 'miko550/CVE-2023-32315: Openfire Console ... - GitHub', 'link': 'https://github.com/miko550/CVE-2023-32315', 'snippet': 'CVE-2023-32315. Openfire Console Authentication Bypass Vulnerability with RCE plugin. Setup. git clone https\xa0...'}, {'title': 'CVE-2023-32315 - CVE', 'link': 'https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2023-32315', 'snippet': 'NOTICE: Transition to the all-new CVE website at WWW.CVE.ORG and CVE Record Format JSON are underway. ... URL:https://github.com/igniterealtime/Openfire/security/\xa0...'}, {'title': 'tangxiaofeng7/CVE-2023-32315-Openfire-Bypass: rce - GitHub', 'link': 'https://github.com/tangxiaofeng7/CVE-2023-32315-Openfire-Bypass', 'snippet': 'rce. Contribute to tangxiaofeng7/CVE-2023-32315-Openfire-Bypass development by creating an account on GitHub.'}, {'title': 'CVE-2023-32315 - NVD', 'link':

In [ ]:

response = router_agent_executor.invoke(
        {"input": f"Given the CVE name: {cve_number}.Follow below steps to give me my answer\
                    2. Path: exploiter/repo/CVE-2021-42013. Extract_type: usage. {task_prompt['readme_output']}\
                    3. Read dockerfile from sec-ai-challenge-hack-main/docker-compose.yml to get the app's mapping port of host machine and its url_link on broswer \
        "}
    )

results = {}
results['steps'] = [action_to_dict(action, message) for action, message in response["intermediate_steps"]]
results['final_output'] = response['output']
print(results)

In [7]:
from langchain_core.agents import _convert_agent_observation_to_messages

In [8]:
??_convert_agent_observation_to_messages

Signature:
_convert_agent_observation_to_messages(
    agent_action: 'AgentAction',
    observation: 'Any',
) -> 'Sequence[BaseMessage]'
Source:   
def _convert_agent_observation_to_messages(
    agent_action: AgentAction, observation: Any
) -> Sequence[BaseMessage]:
    """Convert an agent action to a message.

    This code is used to reconstruct the original AI message from the agent action.

    Args:
        agent_action: Agent action to convert.

    Returns:
        AIMessage that corresponds to the original tool invocation.
    """
    print("agent_action", agent_action, type(agent_action))
    print("observation", observation)
    if isinstance(agent_action, AgentAction):
        return [_create_function_message(agent_action, observation)]
    else:
        return [HumanMessage(content=observation)]
File:      ~/Desktop/epam-repo/llm-security-hackathon/env/lib/python3.11/site-packages/langchain_core/agents.py
Type:      function

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool, StructuredTool, Tool
from langchain_core.callbacks import Callbacks
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [6]:
model = ChatOpenAI(temperature=0, streaming=True)

In [10]:
import random


def where_cat_is_hiding() -> str:
    """Where is the cat hiding right now?"""
    return random.choice(["under the bed", "on the shelf"])

def get_items(place: str) -> str:
    """Use this tool to look up which items are in the given place."""
    if "bed" in place:  # For under the bed
        return "socks, shoes and dust bunnies"
    if "shelf" in place:  # For 'shelf'
        return "books, penciles and pictures"
    else:  # if the agent decides to ask about a different place
        return "cat snacks"

Start


In [9]:
get_items.func.name

AttributeError: 'function' object has no attribute 'name'

In [22]:
tools = [Tool(
            name="find location",
            func=where_cat_is_hiding,
            description="""Where cat hid""",
        ),Tool(
            name="list items",
            func=get_items,
            description="""list items""",
        )]
         


In [18]:
await tool.ainvoke({"place":"kitchen"})

'cat snacks'

In [23]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
# print(prompt.messages) -- to see the prompt
# tools = [get_items, where_cat_is_hiding]
agent = create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}), tools, prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config(
    {"run_name": "Agent"}
)

In [24]:
async for chunk in agent_executor.astream(
    {"input": "what's items are located where the cat is hiding?"}
):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[0].function.name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'tools[0].function.name', 'code': 'invalid_value'}}

In [2]:
from langchain_core.tools import Tool
from exploiter.query.query_text_file import read_read_me, read_dockerfile, clone_repo
from exploiter.query.query_input_content import get_username_and_password, summarize_content

In [11]:
tool = Tool(
            name="Summarize content",
            func=summarize_content,
            description="""useful when you need to summarize content""",
        )

NameError: name 'summarize_content' is not defined

In [4]:
await tool.ainvoke({"information": """ATTENTION Please note that we associated the name Agent with our agent using "run_name"="Agent".\
                                        We'll use that fact later on with the astream_events API."""})

AIMessage(content='- The name "Agent" has been associated with the agent using the parameter "run_name"="Agent".\n- This association will be used later on with the astream_events API.\n- The fact that the name "Agent" is associated with the agent is important for future reference and usage.', response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 69, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-39a8a92b-9969-4a50-91d4-f73681d62b38-0', usage_metadata={'input_tokens': 69, 'output_tokens': 59, 'total_tokens': 128})

In [14]:
tool =Tool(
            name="Information Extraction Agent",
            func=get_username_and_password,
            description="""useful when you need to extract username and password""",
        )

<function __main__.get_items(place: str) -> str>